In [ ]:
import collections
from datetime import datetime
import difflib
import json
import requests
import stanza

STANZA_PIPELINE = stanza.Pipeline('en',
                                  processors='tokenize',)

In [ ]:
url = 'https://en.wikipedia.org/w/api.php'
#title = "Karen_Spärck_Jones_Award"
title = "2008_United_States_presidential_election"
rev_map = collections.defaultdict(list)
for title in [
  #title,
              f'Talk:{title}',]:
  print(title)
  rvstart = None
  while True:
    params = {
    'action':"query",
    'prop':"revisions",
    'titles':title,
    'rvlimit': 500,
    "rvprop":"timestamp|user|comment|ids|content",
    "rvdir": "newer",
    "format":"json"
    }
    if rvstart is not None:
      params['rvstart'] = rvstart
    response = requests.get(url, params=params)
    submissions = response.json()
    if 'batchcomplete' not in submissions:
      rvstart = submissions['continue']['rvcontinue'].split('|')[0]
    for i in list(submissions['query']['pages'].values())[0]['revisions']:
      p = {"title": title}
      p.update(i)
      rev_map[datetime.fromisoformat(i['timestamp'][:-1]).timestamp()].append(p)
    print(datetime.fromtimestamp(max(rev_map.keys())))
    if 'batchcomplete' in submissions:
      break
    

In [ ]:
# i=0

# for t in sorted(rev_map.keys()):
#   for k, v in rev_map[t][0].items():
#     if k in ['*', 'contentformat', 'contentmodel']:
#       continue
#     print(f'{k}: {v}\t', end="")
#   print()
#   print()
#   i += 1
#   if i == 1000:
#     break

In [ ]:
sentence_bank = collections.defaultdict(list)

for k,v in rev_map.items():
  print(k)
  vals = v[0]
  tokenized = STANZA_PIPELINE(vals['*'])
  title = vals['title']
  sentence_indices= []
  for sentence in tokenized.sentences:
    if sentence.text in sentence_bank[title]:
      sentence_indices.append(sentence_bank[title].index(sentence.text))
    else:
      sentence_indices.append(len(sentence_bank[title]))
      sentence_bank[title].append(sentence.text)
  print(len(sentence_indices), len(set(sentence_indices)), len(sentence_bank[title]))

In [ ]:
flag = 0

for k in reversed(sorted(rev_map.keys())):
  v = rev_map[k]
  for i in v:
    if 'edit' in json.dumps(i) and 'request' in json.dumps(i):
      print(i)
      print()
      flag = 1
      break
  if flag:
    break

In [ ]:
import pywikibot as pw
import re
site = pw.Site('en', 'wikipedia')
edit_request_count = 0
for page in site.allpages(prefix='2004_United_States_presidential_election/', namespace='Talk'):
    print(page.title())
    edit_request_count += len(re.findall('Edit.request', page.text))

In [ ]:
edit_request_count